In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
# Premise numbers can have leading zeros, which are lost if pandas reads them in as integers instead of strings
keith_data_df = pd.read_excel(
    r'C:\Users\s346557\Documents\LocalData\KeithFieldData\Meter Interruption Monitoring (MIM).xlsx', 
    dtype={
        'Premise #':str, 
        'IO 52 Order #':str
    }
)
keith_data_df=keith_data_df.dropna(subset=['Premise #'])

In [ ]:
keith_data_df.dtypes

In [ ]:
keith_data_df

In [ ]:
# Probably only want those entries for which the investigation was completed
# NOTE: 'Investigation completed' column, when not None, contains date of investigation
keith_data_df=keith_data_df[keith_data_df['Investigation completed'].notna()]

In [ ]:
keith_data_df.columns.tolist()

In [ ]:
keith_data_df['Date'].nunique()

In [ ]:
keith_data_df['30 day monitoring date'].nunique()

In [ ]:
keith_data_df['30 day monitoring date']-keith_data_df['Date']

In [ ]:
(keith_data_df['30 day monitoring date']-keith_data_df['Investigation completed']).unique()

In [ ]:
pd.Timedelta('30D')

In [ ]:
keith_data_df[keith_data_df['30 day monitoring date']-keith_data_df['Investigation completed']!=pd.Timedelta('30D')]

In [ ]:
keith_data_df[keith_data_df['30 day monitoring date']-keith_data_df['Investigation completed']!=pd.Timedelta('30D')][[
    'Date', '30 day monitoring date', 'Investigation completed'
]]

In [ ]:
# I believe I should grab data for each premise number in a window surrounding 'Investigation completed'
search_window_half_width = pd.Timedelta('30D')

In [ ]:
print(keith_data_df.shape[0])
print(keith_data_df['Investigation completed'].nunique())

In [ ]:
assert(keith_data_df['IO 52 Order #'].nunique()==keith_data_df.shape[0])
daq_df = keith_data_df[['IO 52 Order #', 'Premise #', 'Investigation completed']].copy()
daq_df['t_search_min'] = daq_df['Investigation completed']-search_window_half_width
daq_df['t_search_max'] = daq_df['Investigation completed']+search_window_half_width
daq_df = daq_df.rename(columns={'IO 52 Order #':'IO_52_Order_Nb'})

In [ ]:
#----------
opcos = None
# batch_size=200
batch_size=10
usg_split_to_CTEs=True
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args=None
# addtnl_groupby_cols=None 
addtnl_groupby_cols=['IO_52_Order_Nb']

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
cols_of_interest_ami = TableInfos.AMINonVee_TI.std_columns_of_interest
verbose=True
n_update=1

In [ ]:
#--------------------------------------------------
sql_function_kwargs = dict(
    split_to_CTEs=usg_split_to_CTEs, 
    join_mp_args=False, 
    df_args = dict(
        addtnl_groupby_cols=addtnl_groupby_cols, 
        t_search_min_col='t_search_min', 
        t_search_max_col='t_search_max'
    ), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    save_and_dump=True,  
    sort_coll_to_split=True,
    verbose=verbose, n_update=n_update
)
#----------
if opcos is not None:
    addtnl_sql_function_kwargs = dict(
        build_sql_function_kwargs=dict(opco=opcos)
    )
    sql_function_kwargs = {**sql_function_kwargs, 
                           **addtnl_sql_function_kwargs}
#-------------------------
sql_function_kwargs = Utilities.supplement_dict_with_default_values(
    to_supplmnt_dict=sql_function_kwargs, 
    default_values_dict=dict(
        df_outage=daq_df, 
        batch_size=batch_size, 
        df_args=dict(
            mapping_to_ami={'Premise #':'premise_nbs'}, 
            is_df_consolidated=False
        )
    ), 
    extend_any_lists=True,
    inplace=True
)

In [ ]:
ami_save_args = dict(
    save_to_file=True, 
    save_dir = os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'AMINonVee'), 
    save_name=r'ami_nonvee.csv', 
    index=True
)


end_events_save_args = dict(
    save_to_file=True, 
    save_dir = os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'EndEvents'), 
    save_name=r'end_events.csv', 
    index=True
)

# TODO!!!!!!!!!!!!!
# date_only in build_sql functions!

In [ ]:
# sql_function_kwargs['cols_of_interest'] = cols_of_interest_ami

# start=time.time()
# while True:
#     try:
#         ami_nonvee = AMINonVee(
#             df_construct_type=df_construct_type, 
#             contstruct_df_args = contstruct_df_args, 
#             build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
#             build_sql_function_kwargs=sql_function_kwargs, 
#             init_df_in_constructor=True, 
#             save_args=ami_save_args
#         )

#         break # stop the loop if the function completes sucessfully
#     except Exception as e:
#         print("Function errored out!", e)
#         print("Retrying ... ")
# build_time = time.time()-start
# print(build_time)

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMINonVee,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args, 
        build_sql_function        = AMINonVee_SQL.build_sql_usg_for_outages, 
        build_sql_function_kwargs = sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = ami_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
# sql_function_kwargs['cols_of_interest'] = cols_of_interest_end_dev_event

# start=time.time()
# while True:
#     try:
#         end_events = AMIEndEvents(
#             df_construct_type=df_construct_type, 
#             contstruct_df_args = contstruct_df_args, 
#             build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
#             build_sql_function_kwargs=sql_function_kwargs, 
#             init_df_in_constructor=True, 
#             save_args=end_events_save_args
#         )

#         break # stop the loop if the function completes sucessfully
#     except Exception as e:
#         print("Function errored out!", e)
#         print("Retrying ... ")
# build_time = time.time()-start
# print(build_time)

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMIEndEvents,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args, 
        build_sql_function        = AMIEndEvents_SQL.build_sql_end_events_for_outages, 
        build_sql_function_kwargs = sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = end_events_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
assert(0)

In [ ]:
ami_df = GenAn.read_df_from_csv_dir_batches(
    files_dir=ami_save_args['save_dir'], 
    file_path_glob=r'*.csv', 
    file_path_regex=None, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    drop_unnamed0_col=True, 
    pd_read_csv_kwargs={}, 
    assert_all_cols_equal=True    
)

In [ ]:
ede_df = GenAn.read_df_from_csv_dir_batches(
    files_dir=end_events_save_args['save_dir'], 
    file_path_glob=r'*.csv', 
    file_path_regex=None, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    drop_unnamed0_col=True, 
    pd_read_csv_kwargs={}, 
    assert_all_cols_equal=True    
)

In [ ]:
keith_data_df['Premise #'].nunique()

In [ ]:
ede_df['aep_premise_nb'].nunique()

In [ ]:
ede_df

In [ ]:
ami_df = ami_df.rename(columns={'IO_52_Order_Nb_GPD_FOR_SQL':'IO_52_Order_Nb'})
#-----
ami_df = AMINonVee.perform_std_initiation_and_cleaning(ami_df)
#-----
# Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ami_df, 
    time_col='starttimeperiod', 
    placement_col='starttimeperiod_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)
ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ami_df, 
    time_col='endtimeperiod', 
    placement_col='endtimeperiod_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)

In [ ]:
ede_df = ede_df.rename(columns={'IO_52_Order_Nb_GPD_FOR_SQL':'IO_52_Order_Nb'})
ede_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ede_df, 
    time_col='valuesinterval', 
    placement_col='valuesinterval_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)
ede_df = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df)
#-----
ede_cols_to_keep = [
    'valuesinterval_local', 
    'reason', 
    'serialnumber', 
    'aep_premise_nb', 
    'enddeviceeventtypeid', 
    'event_type', 
    'IO_52_Order_Nb'
]
ede_df = ede_df[ede_cols_to_keep]

In [ ]:
ede_df

In [ ]:
og_shape0 = ede_df.shape[0]
ede_df = pd.merge(
    ede_df, 
    keith_data_df[['Premise #', 'Transformer #', 'Investigation completed', 'IO 52 Order #']].rename(
        columns={
            'Premise #':'aep_premise_nb', 
            'IO 52 Order #':'IO_52_Order_Nb'
        }
    ), 
    how='left', 
    left_on=['aep_premise_nb', 'IO_52_Order_Nb'], 
    right_on=['aep_premise_nb', 'IO_52_Order_Nb']
)
assert(ede_df.shape[0]==og_shape0)
#-----
ede_df['valuesinterval_rel'] = ede_df['valuesinterval_local']-ede_df['Investigation completed']

In [ ]:
og_shape0 = ami_df.shape[0]
ami_df = pd.merge(
    ami_df, 
    keith_data_df[['Premise #', 'Transformer #', 'Investigation completed', 'IO 52 Order #']].rename(
        columns={
            'Premise #':'aep_premise_nb', 
            'IO 52 Order #':'IO_52_Order_Nb'
        }
    ), 
    how='left', 
    left_on=['aep_premise_nb', 'IO_52_Order_Nb'], 
    right_on=['aep_premise_nb', 'IO_52_Order_Nb']
)
assert(ami_df.shape[0]==og_shape0)
#-----
ami_df['starttimeperiod_rel'] = ami_df['starttimeperiod_local']-ami_df['Investigation completed']
ami_df['endtimeperiod_rel']   = ami_df['endtimeperiod_local']-ami_df['Investigation completed']

In [ ]:
ami_df[['aep_derived_uom', 'aep_srvc_qlty_idntfr']].value_counts()

In [ ]:
ami_kwh_df = ami_df[
    (ami_df['aep_derived_uom']=='KWH') & 
    (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
]
#-----
ami_vlt_df = ami_df[
    (ami_df['aep_derived_uom']=='VOLT') & 
    (ami_df['aep_srvc_qlty_idntfr']=='AVG')
]

In [ ]:
ami_kwh_df

In [ ]:
# fig, ax = Plot_General.default_subplots()
# AMINonVee.plot_usage(
#     fig=fig, 
#     ax=ax, 
#     data=ami_kwh_df, 
#     x='starttimeperiod_rel', 
#     y='value', 
#     hue='serialnumber', 
#     data_label='', 
#     title_args=None, 
#     ax_args=None, 
#     xlabel_args=None, 
#     ylabel_args=None, 
#     df_mean=None, 
#     df_mean_col=None, 
#     mean_args=None, 
#     draw_without_hue_also=False, 
#     seg_line_freq=None, 
#     palette='colorblind'
# )

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_kwh_df[ami_kwh_df['serialnumber'].isin(ami_kwh_df['serialnumber'].unique().tolist()[:10])], 
    x='starttimeperiod_rel', 
    y='value', 
    hue='serialnumber', 
    data_label='', 
    title_args=None, 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette='colorblind'
)

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_kwh_df[ami_kwh_df['serialnumber'].isin(ami_kwh_df['serialnumber'].unique().tolist()[:2])], 
    x='starttimeperiod_rel', 
    y='value', 
    hue='serialnumber', 
    data_label='', 
    title_args=None, 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette='colorblind'
)

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_vlt_df[ami_vlt_df['serialnumber'].isin(ami_vlt_df['serialnumber'].unique().tolist()[:2])], 
    x='starttimeperiod_rel', 
    y='value', 
    hue='serialnumber', 
    data_label='', 
    title_args=None, 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette='colorblind'
)

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_vlt_df[ami_vlt_df['serialnumber'].isin(ami_vlt_df['serialnumber'].unique().tolist()[2:4])], 
    x='starttimeperiod_rel', 
    y='value', 
    hue='serialnumber', 
    data_label='', 
    title_args=None, 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette='colorblind'
)

In [ ]:
fig, ax = Plot_General.default_subplots()
AMINonVee.plot_usage(
    fig=fig, 
    ax=ax, 
    data=ami_vlt_df[ami_vlt_df['serialnumber'].isin(ami_vlt_df['serialnumber'].unique().tolist()[4:6])], 
    x='starttimeperiod_rel', 
    y='value', 
    hue='serialnumber', 
    data_label='', 
    title_args=None, 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette='colorblind'
)

In [ ]:
    sql = """
    Select * from default.meter_premise
    WHERE technology_tx='{}'
    ORDER BY RAND()
    LIMIT {}
    """

# Unique meter events by meter type (e.g., KV2c)

In [ ]:
def get_distinct_technology_tx(
    conn_aws=None, 
    **kwargs
):
    r"""
    NOTE: default.meter_premise_hist does not have trsf_pole_nb information.
          Therefore, trsf_pole_nbs can only be selected from those currently active.
    """
    #-------------------------
    if conn_aws is None:
        conn_aws = Utilities.get_athena_prod_aws_connection()
    #-------------------------
    kwargs = copy.deepcopy(kwargs)
    _ = kwargs.pop('addtnl_cols_of_interest', None)
    #-------------------------
    kwargs['technology_tx_col'] = kwargs.get('technology_tx_col', 'technology_tx')
    #-------------------------
    sql = MeterPremise.build_sql_meter_premise(
        cols_of_interest=[kwargs['technology_tx_col']], 
        **kwargs
    )
    sql.sql_select.select_distinct=True
    #-------------------------
    df = pd.read_sql_query(sql.get_sql_statement(), conn_aws)
    #-------------------------
    return df

In [ ]:
def get_random_MPs_of_tech_type(
    technology_tx_i, 
    n_PNs, 
    opcos=None, 
    conn_aws=None, 
    return_sql=False, 
    **kwargs
):
    r"""
    Randomly grab MeterPremise (MP) entries of technology_tx type
    
    kwargs:
        Can be used to input any sql kwargs for MeterPremise (e.g., state requirements, etc.)
    """
    #-------------------------
    if conn_aws is None:
        conn_aws = Utilities.get_athena_prod_aws_connection()
    #-------------------------
    if kwargs:
        kwargs['technology_tx']     = technology_tx_i
        kwargs['technology_tx_col'] = kwargs.get('technology_tx_col', 'technology_tx')
    else:
        kwargs = dict(
            technology_tx=technology_tx_i, 
            technology_tx_col='technology_tx'
        )
    #-----
    # kwargs['alias'] = kwargs.get('alias', 'MP')
    #-----
    # technology_tx needs to be included in cols_of_interest 
    if 'addtnl_cols_of_interest' in kwargs.keys():
        if kwargs['technology_tx_col'] not in kwargs['addtnl_cols_of_interest']:
            kwargs['addtnl_cols_of_interest'].append(kwargs['technology_tx_col'])
    else:
        kwargs['addtnl_cols_of_interest'] = [kwargs['technology_tx_col']]
    #-------------------------
    sql = MeterPremise.build_sql_meter_premise(**kwargs)
    #-------------------------
    # Randomize order
    sql.set_sql_orderby(SQLOrderBy(field_descs=['RAND()']))
    #-------------------------
    # Limit number of entries returned
    if n_PNs is not None:
        sql.set_limit(n_PNs)
    #-------------------------
    if opcos is not None:
        sql_stmnt = MeterPremise.add_opco_nm_to_mp_sql(
            mp_sql=sql, 
            opcos=opcos, 
            comp_cols=['opco_nm'], 
            comp_alias='COMP', 
            join_type='LEFT'
        )
    else:
        sql_stmnt = sql.get_sql_statement()
    #-------------------------
    df = pd.read_sql_query(sql_stmnt, conn_aws)
    if df.shape[0]>0:
        assert(
            (df[kwargs['technology_tx_col']].nunique()==1) and
            (df[kwargs['technology_tx_col']].unique()[0]==technology_tx_i)
        )
    #-------------------------
    if return_sql:
        return df, sql_stmnt
    else:
        return df

def get_random_PNs_of_tech_type(
    technology_tx_i, 
    n_PNs, 
    opcos=None, 
    conn_aws=None, 
    return_sql=False, 
    **kwargs
):
    r"""
    """
    #-------------------------
    if conn_aws is None:
        conn_aws = Utilities.get_athena_prod_aws_connection()
    #-------------------------
    mp_df, sql_stmnt = get_random_MPs_of_tech_type(
        technology_tx_i=technology_tx_i, 
        n_PNs=n_PNs, 
        opcos=opcos, 
        conn_aws=conn_aws, 
        return_sql=True, 
        **kwargs
    )
    #-------------------------
    if mp_df.shape[0]==0:
        return_PNs = []
    else:
        return_PNs = mp_df['prem_nb'].unique().tolist()
    #-------------------------
    if return_sql:
        return return_PNs, sql_stmnt
    else:
        return return_PNs.tolist() 

In [ ]:
def get_random_PNs_for_each_tech_type(
    n_PNs, 
    opcos=None, 
    conn_aws=None, 
    return_sql=False, 
    **kwargs
):
    r"""
    """
    #-------------------------
    if conn_aws is None:
        conn_aws = Utilities.get_athena_prod_aws_connection()
    #-------------------------
    kwargs['technology_tx_col'] = kwargs.get('technology_tx_col', 'technology_tx')
    #-------------------------
    distinct_technology_txs = get_distinct_technology_tx(
        conn_aws=conn_aws, 
        **kwargs
    )
    #-------------------------
    return_dict = dict()
    return_sql_dict = dict()
    for technology_tx_i in distinct_technology_txs[kwargs['technology_tx_col']].tolist():
        # Skip if blank/empty
        if not technology_tx_i:
            continue
        #-------------------------
        PNs_i, sql_stmnt_i = get_random_PNs_of_tech_type(
            technology_tx_i=technology_tx_i, 
            n_PNs=n_PNs, 
            opcos=opcos, 
            conn_aws=conn_aws, 
            return_sql=True, 
            **kwargs
        )
        #-------------------------
        assert(technology_tx_i not in return_dict.keys())
        assert(technology_tx_i not in return_sql_dict.keys())
        return_dict[technology_tx_i]     = PNs_i
        return_sql_dict[technology_tx_i] = sql_stmnt_i 
    #-------------------------
    if return_sql:
        return return_dict, return_sql_dict
    else:
        return return_dict

In [ ]:
def get_distinct_events_by_meter_type(
    n_PNs_to_sample, 
    date_0, 
    date_1, 
    rand_PNs_for_tech_txs_dict=None, 
    batch_size=10000, 
    return_full_dfs=False, 
    opcos=None, 
    conn_aws=None, 
    **kwargs
):
    r"""
    opcos:
        Only used if rand_PNs_for_tech_txs_dict is None (i.e., only used when
          random PNs are found within this function, not supplied by user)
    """
    #-------------------------
    if rand_PNs_for_tech_txs_dict is None:
        rand_PNs_for_tech_txs_dict = get_random_PNs_for_each_tech_type(
            n_PNs=n_PNs_to_sample, 
            opcos=opcos, 
            conn_aws=conn_aws, 
            return_sql=False, 
            **kwargs
        )
    #-------------------------
    return_dict = dict()
    for tech_tx_i,PNs_i in rand_PNs_for_tech_txs_dict.items():
        #----------
        if len(PNs_i)==0:
            if return_full_dfs:
                return_dict[tech_tx_i] = pd.DataFrame()
            else:
                return_dict[tech_tx_i] = pd.Series()
            continue
        #----------
            
        ede_i = AMIEndEvents(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args=None, 
            init_df_in_constructor=True, 
            build_sql_function=AMIEndEvents_SQL.build_sql_end_events, 
            build_sql_function_kwargs=dict(
                date_range = [date_0, date_1], 
                premise_nbs = PNs_i, 
                field_to_split='premise_nbs', 
                batch_size=batch_size, 
                verbose=True
            ), 
            save_args=False, 
            **kwargs
        )
        #-------------------------
        ede_df_i = ede_i.df.copy()
        if ede_df_i.shape[0]>0:
            ede_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(
                df=ede_df_i, 
                reason_col='reason', 
                placement_col='reason'
            )
        #-------------------------
        assert(tech_tx_i not in return_dict.keys())
        if return_full_dfs:
            return_dict[tech_tx_i] = ede_df_i
        else:
            return_dict[tech_tx_i] = ede_df_i['reason'].value_counts()
    #-------------------------
    return return_dict

In [ ]:
def plot_reasons_by_meter_type(
    fig_num, 
    reasons_by_meter_type_df, 
    n_per_plot=10, 
    orient='v'
):
    r"""
    """
    #-------------------------
    assert(orient=='h' or orient=='v')
    if orient=='v':
        x='variable' 
        y='value'
    else:
        y='variable' 
        x='value'
    #-------------------------
    batch_idxs = Utilities.get_batch_idx_pairs(
        n_total=reasons_by_meter_type_df.shape[1],
        batch_size=n_per_plot,
        absorb_last_pair_pct=0.5,
    )
    #-------------------------
    n_plots = len(batch_idxs)
    fig_ax_pairs = []
    for i_plot, batch_i in enumerate(batch_idxs):
        fig_i, ax_i = Plot_General.default_subplots(
            fig_num=fig_num, 
            unit_figsize_width=14,
            unit_figsize_height=10,
        )
        #-----
        i_beg = batch_i[0]
        i_end = batch_i[1]
        order = reasons_by_meter_type_df.columns[i_beg:i_end].tolist()
        #-----
        sns.barplot(
            ax=ax_i, 
            data=reasons_by_meter_type_df.iloc[:, i_beg:i_end].reset_index().melt(id_vars='meter_type'), 
            x=x, 
            y=y, 
            hue='meter_type', 
            order=order, 
            orient=orient
        )
        #-----
        if orient=='v':
            xtick_rename_dict = {x_tick_i:i+1 for i,x_tick_i in enumerate(order)}
            Plot_General.set_general_plotting_args(
                ax=ax_i, 
                ax_args=dict(xticklabels=list(xtick_rename_dict.values()))
            )
            subplot_layout_params = Plot_General.get_subplot_layout_params(fig_i)
            Plot_General.generate_xtick_labels_legend_textbox(
                fig=fig_i, 
                xtick_rename_dict=xtick_rename_dict, 
                text_x_pos=1.02*subplot_layout_params['right'], 
                text_y_pos=subplot_layout_params['top'], 
                fontsize=12
            )
        #-----
        fig_ax_pairs.append((fig_i, ax_i))
        fig_num+=1
    return fig_num, fig_ax_pairs

In [ ]:
conn_aws = Utilities.get_athena_prod_aws_connection()

# BEG DEL

In [ ]:
import MeterPremise
reload(MeterPremise)
from MeterPremise import MeterPremise

In [ ]:
a,b = get_random_MPs_of_tech_type(
    technology_tx_i='AMI', 
    n_PNs=10, 
    opcos='oh', 
    conn_aws=conn_aws, 
    return_sql=True, 
)

In [ ]:
print(b)

In [ ]:
a,b = get_random_PNs_of_tech_type(
    technology_tx_i = 'AMI', 
    n_PNs=10, 
    opcos='oh', 
    conn_aws=conn_aws, 
    return_sql=True, 
)

In [ ]:
print(b)

In [ ]:
a

# END DEL

In [ ]:
n_PNs=100000
# n_PNs=10
conn_aws=None
kwargs = dict()
# kwargs = dict(technology_tx_col='devc_cd')

date_0 = pd.to_datetime('2022-01-01')
date_1 = pd.to_datetime('2023-09-01')

return_full_dfs=False

In [ ]:
rand_PNs_for_tech_txs_dict, sqls_dict = get_random_PNs_for_each_tech_type(
    n_PNs=n_PNs, 
    conn_aws=conn_aws, 
    return_sql=True, 
    **kwargs
)

In [ ]:
for k,v in rand_PNs_for_tech_txs_dict.items():
    print(k)
    print(len(v))
    print()

In [ ]:
for k,v in sqls_dict.items():
    print(k)
    print(v)
    print()

In [ ]:
distinct_events_by_meter_type = get_distinct_events_by_meter_type(
    n_PNs_to_sample=n_PNs, 
    date_0=date_0, 
    date_1=date_1, 
    rand_PNs_for_tech_txs_dict=rand_PNs_for_tech_txs_dict, 
    return_full_dfs=False, 
    conn_aws=None, 
    **kwargs
)

In [ ]:
for tech_tx_i,vcs_i in distinct_events_by_meter_type.items():
    print(tech_tx_i)
    print(vcs_i)
    print('\n\n\n')

In [ ]:
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'distinct_events_by_meter_type.pkl'), 'wb') as handle:
    pickle.dump(distinct_events_by_meter_type, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-----
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'rand_PNs_for_tech_txs_dict.pkl'), 'wb') as handle:
    pickle.dump(rand_PNs_for_tech_txs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-----
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'sqls_dict.pkl'), 'wb') as handle:
    pickle.dump(sqls_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'distinct_events_by_meter_type.pkl'), 'rb') as handle:
    distinct_events_by_meter_type = pickle.load(handle)
#-----
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\KeithFieldData', 'rand_PNs_for_tech_txs_dict.pkl'), 'rb') as handle:
    rand_PNs_for_tech_txs_dict = pickle.load(handle)

In [ ]:
n_PNs_per_meter_type_dict = {k:len(v) for k,v in rand_PNs_for_tech_txs_dict.items()}
n_PNs_per_meter_type = pd.Series(n_PNs_per_meter_type_dict).to_frame(name='n_PNs')
n_PNs_per_meter_type=n_PNs_per_meter_type.sort_values(by='n_PNs', ascending=False)

In [ ]:
n_PNs_per_meter_type

In [ ]:
fig,ax = Plot_General.default_subplots()
n_PNs_per_meter_type.reset_index().plot.scatter(ax=ax, x='index', y='n_PNs')
Plot_General.rotate_ticklabels(
    ax=ax, 
    rotation=90, 
    axis='x'
)

In [ ]:
fig,ax = Plot_General.default_subplots()
n_PNs_per_meter_type.reset_index().plot.scatter(ax=ax, x='index', y='n_PNs')
Plot_General.rotate_ticklabels(
    ax=ax, 
    rotation=90, 
    axis='x'
)
ax.set_yscale('log')

In [ ]:
remove_meter_types_with_no_events=True

# Build reasons_by_meter_type_df and reasons_by_meter_type_df_norm
#-------------------------
# First, find the set of unique reasons from all meter types
all_reasons = []
for meter_type_i, counts_srs_i in distinct_events_by_meter_type.items():
    all_reasons.extend(counts_srs_i.index.tolist())
all_reasons = list(set(all_reasons))
#-------------------------
# Now, essentially convert distinct_events_by_meter_type from a dict to a DataFrame
reasons_by_meter_type_df = pd.DataFrame(columns=all_reasons)
for meter_type_i, counts_srs_i in distinct_events_by_meter_type.items():
    counts_srs_i = counts_srs_i.to_frame().T
    counts_srs_i.index = [meter_type_i]
    reasons_by_meter_type_df = pd.concat([reasons_by_meter_type_df, counts_srs_i])
reasons_by_meter_type_df=reasons_by_meter_type_df.fillna(0)
reasons_by_meter_type_df.index.name = 'meter_type'
#-------------------------
# Let's sort the columns by total number of counts across all meter types
cols_order = reasons_by_meter_type_df.sum().sort_values(ascending=False).index.tolist()
reasons_by_meter_type_df = reasons_by_meter_type_df[cols_order]
#-------------------------
# Find the meter types which did not register any events and remove from reasons_by_meter_type_df
meter_types_wo_events = reasons_by_meter_type_df.sum(axis=1)[reasons_by_meter_type_df.sum(axis=1)==0].index.tolist()
if remove_meter_types_with_no_events:
    reasons_by_meter_type_df=reasons_by_meter_type_df.drop(index=meter_types_wo_events)
#-------------------------
# Normalize to build reasons_by_meter_type_df_norm
reasons_by_meter_type_df_norm = reasons_by_meter_type_df.div(reasons_by_meter_type_df.sum(axis=1), axis='rows')
if not remove_meter_types_with_no_events:
    # In case there were any divisions by 0 above!
    reasons_by_meter_type_df_norm=reasons_by_meter_type_df_norm.fillna(0)

In [ ]:
reasons_by_meter_type_df_norm

In [ ]:
fig,ax = Plot_General.default_subplots()
sns.barplot(
    ax=ax, 
    data=reasons_by_meter_type_df_norm.iloc[:, :10].reset_index().melt(id_vars='meter_type'), 
    x='variable', 
    y='value', 
    hue='meter_type',  
)
Plot_General.rotate_ticklabels(
    ax=ax, 
    rotation=90, 
    axis='x'
)

In [ ]:
fig_num=0
n_per_plot = 10

In [ ]:
fig_num, fig_ax_pairs = plot_reasons_by_meter_type(
    fig_num=fig_num, 
    reasons_by_meter_type_df=reasons_by_meter_type_df_norm, 
    n_per_plot=n_per_plot, 
    orient='v'
)

In [ ]:
fig_num, fig_ax_pairs = plot_reasons_by_meter_type(
    fig_num=fig_num, 
    reasons_by_meter_type_df=reasons_by_meter_type_df_norm, 
    n_per_plot=n_per_plot, 
    orient='h'
)

# Run By OPCO

In [ ]:
n_PNs=100000
conn_aws=None
kwargs = dict()
# kwargs = dict(technology_tx_col='devc_cd')

date_0 = pd.to_datetime('2022-01-01')
date_1 = pd.to_datetime('2023-09-01')

return_full_dfs=False

opcos = ['ap', 'ky', 'oh', 'im', 'pso', 'swp', 'tx']
base_dir = r'C:\Users\s346557\Documents\LocalData\KeithFieldData\_ByOPCO'

In [ ]:
for opco in opcos:
    print(opco)
    rand_PNs_for_tech_txs_dict_i, sqls_dict_i = get_random_PNs_for_each_tech_type(
        n_PNs=n_PNs, 
        opcos=[opco], 
        conn_aws=conn_aws, 
        return_sql=True, 
        **kwargs
    )
    #-----
    distinct_events_by_meter_type_i = get_distinct_events_by_meter_type(
        n_PNs_to_sample=n_PNs, 
        date_0=date_0, 
        date_1=date_1, 
        rand_PNs_for_tech_txs_dict=rand_PNs_for_tech_txs_dict_i, 
        return_full_dfs=False, 
        opcos=[opco], 
        conn_aws=None, 
        **kwargs
    )
    #-----
    save_dir_i = os.path.join(base_dir, opco)
    if not os.path.exists(save_dir_i):
        os.makedirs(save_dir_i)
    with open(os.path.join(save_dir_i, 'distinct_events_by_meter_type.pkl'), 'wb') as handle:
        pickle.dump(distinct_events_by_meter_type_i, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #-----
    with open(os.path.join(save_dir_i, 'rand_PNs_for_tech_txs_dict.pkl'), 'wb') as handle:
        pickle.dump(rand_PNs_for_tech_txs_dict_i, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #-----
    with open(os.path.join(save_dir_i, 'sqls_dict.pkl'), 'wb') as handle:
        pickle.dump(sqls_dict_i, handle, protocol=pickle.HIGHEST_PROTOCOL)